In [1]:
!pip install transformers
#!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
######test##############
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
#from langdetect import detect
#from langdetect import detect_langs
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 导入数据

In [19]:
df_datatopic=pd.read_csv("/content/drive/MyDrive/情感打分+情感轨道/情感打分数据包/japanese_txt.csv")
df_datatopic=df_datatopic.iloc[:200,:]
df_datatopic

,txt,year,text_length,txt_seg,Topic,Doc_ID
0,千葉市は三十日までに、新型肺炎（重症急性呼吸器症候群＝ＳＡＲＳ）の影響で、中国にある同市の友...,2003.07.01,196,千葉 市 三 十 日 新型 肺炎 重症 急性 呼吸 器 症候群 ＳＡＲＳ 影響 中国 ある ...,24,9
1,中国・広東省深セン市を拠点に、国内はもとより、香港などで活躍するファッションデザイナー、李艶...,1991.02.05,461,中国 広東 省 深セン 市 拠点 国内 もとより 香港 活躍 する ファッションデザイナー ...,255,12
2,野球の国・地域別対抗戦、第３回ワールド・ベースボール・クラシック（ＷＢＣ）の福岡１次ラウンド...,2013.03.04,229,野球 国 地域 別 対抗 戦 第 回 ワールド ベースボール クラシック ＷＢＣ 福岡 次 ...,24,13
3,◇Ｌｏｎｄｏｎ２０１２７月に開幕するロンドン五輪に出場するバドミントン・男子シングルスの佐々...,2012.05.10,680,Ｌｏｎｄｏｎ ７月 開幕 する ロンドン 五輪 出場 する バドミントン 男子 シングルス ...,24,16
4,１０日午前３時１５分ごろ、大田市・大岬灯台の北西約１５キロの日本海で、大田市漁協所属の底引き...,2005.03.11,255,日 午前 時 分 ごろ 大田 市 大岬 灯台 北西 約 キロ 日本海 大田 市 漁協 所属 ...,277,17
...,...,...,...,...,...,...
195,東京マラソン２０１４（読売新聞社など共催）は２３日、東京都庁から東京ビッグサイトまでの４２・...,2014.02.21,694,東京 マラソン 読売新聞社 共催 日 東京 都庁 東京 ビッグ サイト キロ 行わ れる 前...,24,503
196,◇単位＝円◆米ドル（銀行間直物）午後５時現在１１３．３５～１１３．３８前営業日比△１．８２高...,2007.09.11,326,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,18,504
197,【マニラ２４日＝坂元隆】クリントン米大統領のアジア歴訪に同行しているウィンストン・ロード国務...,1996.11.25,354,マニラ 日 坂元 隆 クリントン 米 大統領 アジア 歴訪 同行 し いる ウィンストン・ロ...,87,515
198,◇こうしろ・かずよし（横浜国立大学教授）昨年夏の中東紛争と金融引き締めを境に、人手不足もやや...,1991.01.16,694,こう しろ かずよ し 横浜国立大学 教授 昨年 夏 中東 紛争 金融 引き締め 境 人手 ...,182,516


In [ ]:
#japanese_txt_index=[]
#chinese_txt_index=[]
#for i in range(len(df_datatopic)):
  #sentence=str(df_datatopic.iloc[i,0])
  #print(sentence)
  #if detect(sentence)=="ja":
    #japanese_txt_index.append(i)
  #else:
    #chinese_txt_index.append(i)
#df_japanese=df_datatopic.iloc[japanese_txt_index,:]
#df_chinese=df_datatopic.iloc[chinese_txt_index,:]
#df_chinese

Output hidden; open in https://colab.research.google.com to view.

简单看一下数据类型


In [ ]:
#excel 使用 pd.read_excel; csv 用 pd.read.csv
#df = pd.read_csv("/content/drive/MyDrive/情感打分数据包/datatopicid5w2.csv")
#df

,txt,year,text_length,txt_seg,Topic,Doc_ID
0,韩健和普拉卡什获男子单打决赛权据新华社东京1月20日电中国选手韩健和印度选手普拉卡什，今天在...,1983.01.22,365,韩健 和 普拉卡什 获 男子单打 决赛权 据 新华社 东京 月 20 日电 中国 选手 韩健...,24,3
1,本报讯记者罗雪村报道：4月25日上午，曾经亲历60年前冀中“五一反扫荡”的600余位老战士在...,2002.04.27,366,本报讯 记者 罗雪村 报道 月 25 日 上午 曾经 亲历 60 年前 冀中 五一 反 扫荡...,228,4
2,张月华：一位企业家的雷锋情结夜幕下，公路旁，一辆摩托车横卧地面，一男一女躺在血泊中。“救救我...,2002.06.12,692,张 月华 一位 企业家 的 雷锋 情结 夜幕 下 公路 旁 一辆 摩托车 横卧 地面 一男一...,94,5
3,千葉市は三十日までに、新型肺炎（重症急性呼吸器症候群＝ＳＡＲＳ）の影響で、中国にある同市の友...,2003.07.01,196,千葉 市 三 十 日 新型 肺炎 重症 急性 呼吸 器 症候群 ＳＡＲＳ 影響 中国 ある ...,24,9
4,中国・広東省深セン市を拠点に、国内はもとより、香港などで活躍するファッションデザイナー、李艶...,1991.02.05,461,中国 広東 省 深セン 市 拠点 国内 もとより 香港 活躍 する ファッションデザイナー ...,255,12
...,...,...,...,...,...,...
49995,◆プリムラ・マラコイデス（サクラソウ科）日本にもサクラソウとクリンソウの二つの原種があります...,2000.01.18,648,プリムラ・マラコイデス サクラソウ 科 日本 サクラソウ クリンソウ 二つ 原種 あり ます...,204,91153
49996,◇単位＝円◆米ドル（銀行間直物）午後５時現在７９．９６〜７９．９８前営業日比△０．１８高値７...,2012.10.27,385,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,7,91159
49997,８月６日（土）午後１時〜３時、太閤園ガーデンホール（大阪市都島区網島町９の１０）伝統琵琶奏者...,2011.06.09,272,８月 日 土 午後 時 時 太閤園 ガーデン ホール 大阪 市 都島 区 網島 町 伝統 琵...,174,91160
49998,高円宮杯・第五十二回全日本中学校英語弁論大会（読売新聞社など主催）の中部日本地区代表を選ぶ決...,2000.11.25,169,高円宮 杯 第 五 十 二 回 全日本 中学校 英語 弁論 大会 読売新聞社 主催 中部 日...,109,91161


## 定义模型

In [5]:
device = torch.device("cpu")

In [6]:
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"#"twitter-xlm-roberta-base-sentiment" #运行时，会直接在网上下载模型，如果报错，可以自己去hugging face上下载好模型，这里改成本地地址

In [7]:
!pip install --no-cache-dir transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.8 MB/s 


In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 定义打分函数

In [17]:
# function to obtain the sentiment of the headline using textblob package
def get_polarity(text):
    try:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        #print(output)
        scores = output[0][0].detach().numpy()
        #print(scores)
        scores = softmax(scores)
        #print(scores)
        neg = scores[0]
        neu = scores[1]
        pos = scores[2] 
    except:
        neg = 0.0
        neu = 0.0
        pos = 0.0
    return neg, pos

In [20]:
#得到三种感情倾向的得分（消极、中性、积极）
df_datatopic["Score_Negative"], df_datatopic["Score_Neutral"], df_datatopic["Score_Positive"]=zip(*df_datatopic['txt_seg'].apply(get_polarity))
#df_datatopic["Score_Negative"], df_datatopic["Score_Positive"]=zip(*df_datatopic['txt_seg'].apply(get_polarity))
df_datatopic

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1321,  0.0813]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-0.13209194  0.08130327]
[0.4468527  0.55314726]
SequenceClassifierOutput(loss=None, logits=tensor([[0.0111, 0.0480]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[0.01113334 0.04804041]
[0.49077424 0.5092257 ]
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0460,  0.0082]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-0.04596542  0.00815279]
[0.48647374 0.5135262 ]
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0258,  0.0458]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-0.02580049  0.04583448]
[0.4820989 0.5179011]
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0277, -0.0378]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-0.027737   -0.03780953]
[0.5025181  0.49748182]
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0533,  0.0141]], grad

,txt,year,text_length,txt_seg,Topic,Doc_ID,Score_Negative,Score_Positive
0,千葉市は三十日までに、新型肺炎（重症急性呼吸器症候群＝ＳＡＲＳ）の影響で、中国にある同市の友...,2003.07.01,196,千葉 市 三 十 日 新型 肺炎 重症 急性 呼吸 器 症候群 ＳＡＲＳ 影響 中国 ある ...,24,9,0.446853,0.553147
1,中国・広東省深セン市を拠点に、国内はもとより、香港などで活躍するファッションデザイナー、李艶...,1991.02.05,461,中国 広東 省 深セン 市 拠点 国内 もとより 香港 活躍 する ファッションデザイナー ...,255,12,0.490774,0.509226
2,野球の国・地域別対抗戦、第３回ワールド・ベースボール・クラシック（ＷＢＣ）の福岡１次ラウンド...,2013.03.04,229,野球 国 地域 別 対抗 戦 第 回 ワールド ベースボール クラシック ＷＢＣ 福岡 次 ...,24,13,0.486474,0.513526
3,◇Ｌｏｎｄｏｎ２０１２７月に開幕するロンドン五輪に出場するバドミントン・男子シングルスの佐々...,2012.05.10,680,Ｌｏｎｄｏｎ ７月 開幕 する ロンドン 五輪 出場 する バドミントン 男子 シングルス ...,24,16,0.000000,0.000000
4,１０日午前３時１５分ごろ、大田市・大岬灯台の北西約１５キロの日本海で、大田市漁協所属の底引き...,2005.03.11,255,日 午前 時 分 ごろ 大田 市 大岬 灯台 北西 約 キロ 日本海 大田 市 漁協 所属 ...,277,17,0.482099,0.517901
...,...,...,...,...,...,...,...,...
195,東京マラソン２０１４（読売新聞社など共催）は２３日、東京都庁から東京ビッグサイトまでの４２・...,2014.02.21,694,東京 マラソン 読売新聞社 共催 日 東京 都庁 東京 ビッグ サイト キロ 行わ れる 前...,24,503,0.000000,0.000000
196,◇単位＝円◆米ドル（銀行間直物）午後５時現在１１３．３５～１１３．３８前営業日比△１．８２高...,2007.09.11,326,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,18,504,0.438977,0.561023
197,【マニラ２４日＝坂元隆】クリントン米大統領のアジア歴訪に同行しているウィンストン・ロード国務...,1996.11.25,354,マニラ 日 坂元 隆 クリントン 米 大統領 アジア 歴訪 同行 し いる ウィンストン・ロ...,87,515,0.489591,0.510409
198,◇こうしろ・かずよし（横浜国立大学教授）昨年夏の中東紛争と金融引き締めを境に、人手不足もやや...,1991.01.16,694,こう しろ かずよ し 横浜国立大学 教授 昨年 夏 中東 紛争 金融 引き締め 境 人手 ...,182,516,0.000000,0.000000


## 保持文件

In [21]:
df_datatopic.to_excel("/content/drive/MyDrive/情感打分+情感轨道/情感打分数据包/日文/日文demo.xlsx", index=False)